In [171]:
!git clone https://github.com/BrooksLabUCSC/flair.git
!pip install kerneltree

Cloning into 'flair'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 628 (delta 1), reused 5 (delta 1), pack-reused 617
Receiving objects: 100% (628/628), 19.27 MiB | 6.89 MiB/s, done.
Resolving deltas: 100% (316/316), done.
Checking connectivity... done.
Checking out files: 100% (44/44), done.


In [107]:
import numpy as np
import csv
import pandas as pd
import matplotlib.pyplot as plt

**Script for Flair. 1) Converts bam mapped reads file to bed12 and corrects misaligned splice sites using a genome annotation. 2) Collapse corrected reads into high-confidence isoforms and quantifies them by counting converted into tpm.**

In [34]:
def Flair(
    main_dir, #full path of working directory
    bam_input, #only filename, path same as main_dir
    genome,    #full path of reference genome (in ref_fasta folder)
    chr_sizes, #full path to tab-delim file of chromosome sizes, names must correspond to both annotation and reference
    chr_annot, #full path to gtf annotation file
    reads,     #full path to fa file of UN-mapped raw reads
    file_org): #full path to tab-delim file listing sample information
    
    !mkdir {main_dir}/nc.flair
#     !samtools faidx {genome}  <- if using a new genome fasta file
    print("Copy paste into command line window")
    print("1 convert, correct, collapse scripts:") ##**NB: module tqdm only installed on python3, not just python
    #both samtools and minimap2 directories are located in software/flair/
        #-s parameter is minimum number of supporting reads for an isoform (default: 3) 
        #--stringent
    print('$python3 software/flair/bin/bam2Bed12.py -i '+main_dir+'/'+bam_input+' > '+ main_dir+'/nc.flair/_clean_sorted.bed12;python3 software/flair/flair.py correct -c '+chr_sizes+' -g '+genome+' -q '+main_dir+'/nc.flair/_clean_sorted.bed12 -o '+main_dir+'/nc.flair/flair -f '+chr_annot+'; time python3 software/flair/flair.py collapse -g '+genome+' -r '+reads+' -f '+chr_annot+' -o '+main_dir+"/nc.flair/flair.collapse -q "+main_dir+'/nc.flair/flair_all_corrected.psl')
    main_dir=main_dir+'/nc.flair'
    print "2 quantify:"
    print 'python3 software/flair/flair.py quantify --tpm -r '+file_org+' -o '+main_dir+'/counts_matrix.tsv -i '+main_dir+'/flair.collapse.isoforms.fa'

In [73]:
Flair(
    main_dir='2019-8-6_cdna/a431.b1.merged',
    bam_input='_cleansort.bam',
    genome='software/ref_fasta/homo_GRCh38_trimmed_ref.fa',
    chr_sizes='software/bedtools2/genomes/clean.human.hg38.genome.chrom.sizes.tsv',
    chr_annot='software/flair/new.gencode.v29.annotation.gtf',
    reads='2019-8-6_cdna/a431.b1/a431.b1.fasta',   ##use raw, unaligned reads
    file_org='2019-7-26_cdna.txt'
)

mkdir: cannot create directory ‘2019-8-6_cdna/a431.b1.merged/nc.flair’: File exists
Copy paste into command line window
1 convert, correct, collapse scripts:
$python3 software/flair/bin/bam2Bed12.py -i 2019-8-6_cdna/a431.b1.merged/_cleansort.bam > 2019-8-6_cdna/a431.b1.merged/nc.flair/_clean_sorted.bed12;python3 software/flair/flair.py correct -c software/bedtools2/genomes/clean.human.hg38.genome.chrom.sizes.tsv -g 2019-6-13_A431/GRCh38_trimmed.fa -q 2019-8-6_cdna/a431.b1.merged/nc.flair/_clean_sorted.bed12 -o 2019-8-6_cdna/a431.b1.merged/nc.flair/flair -f software/flair/new.gencode.v29.annotation.gtf; time python3 software/flair/flair.py collapse -g 2019-6-13_A431/GRCh38_trimmed.fa -r 2019-8-6_cdna/a431.b1/a431.b1.fasta -f software/flair/new.gencode.v29.annotation.gtf -o 2019-8-6_cdna/a431.b1.merged/nc.flair/flair.collapse -q 2019-8-6_cdna/a431.b1.merged/nc.flair/flair_all_corrected.psl
2 quantify:
python3 software/flair/flair.py quantify --tpm -r 2019-8-6_cdna/a431.b1.merged/nc.flair

In [13]:
##Isoform productivity: append label to end of each line of PSL
    ##0: functional stop codon (productive)
    ##1: premature stop codon (unproductive)
    ##2: lncRNA (no start codon) 
    
    #From script: 'Unproductive proportion estimate: unproductive / (valid_transcripts + unproductive))
    ##Single-exon transcripts are counted as lncRNAs.

def IsoformProductivity(directory,
                       reads,
                       annot,
                       genome):
    reads=directory+reads
    outfile=directory+'/isoforms.productivity.psl'
#     outfile=directory+'/isosproductive.psl'
    !python software/flair/bin/mark_productivity.py {reads} {annot} {genome} > {outfile}

    f=open(outfile)
    reader=csv.reader(f,delimiter='\t')

    good_stop=0
    bad_stop=0
    no_start=0
    single_exon=0
    for row in reader:
        n=int(row[-1])
        if n==0: good_stop+=1
        elif n==1: bad_stop+=1
        elif n==2: no_start+=1
        if int(row[17])==1: single_exon+=1

    print directory        
    print 'Productive isoforms: '+str(good_stop)
    print 'Unproductive isoforms: '+str(bad_stop)
    total=good_stop+bad_stop+no_start
    print 'Proportion of productive: ' +str(good_stop*100 / (total))+'%'
    print 'lncRNA/other: '+str(no_start - single_exon)
    print 'Single exon transcripts: '+str(single_exon)
    Productive_isos(directory,outfile)

def Productive_isos(directory,
                   infile):
    out=directory+'/all_productive_isos.psl'
    i=open(infile,'r')
    o=open(out,'a')
    reader=csv.reader(i,delimiter='\t')
    writer = csv.writer(o,delimiter='\t')
    for row in reader:
        length=abs(int(row[16])-int(row[15]))
        if int(row[-1])==2 and length>=300: writer.writerow(row)
        elif int(row[21])==0: writer.writerow(row)   

##Marks each isoform by appending new column to each entry of PSL
    ##0: spliced
    ##1: intron-retaining
    
def IntronRetention(directory,
                   isoforms='exonsgenes.collapse.isoforms.psl',
                   outfile='isoforms.ir.psl'):
    isoforms=directory+'/'+isoforms 
    outfile=directory+'/'+outfile
    coords=directory+'/coords.txt'
    !python software/flair/bin/mark_intron_retention.py {isoforms} {outfile} {coords}

    ##Count intron-retaining and spliced isoforms
    f=open(outfile)
    reader=csv.reader(f,delimiter='\t')
    spliced=0
    intron_retain=0
    splice_pos=0
    splice_neg=0
    retain_pos=0
    retain_neg=0
    for row in reader:
        if int(row[21])==0: spliced+=1
        elif int(row[21])==1: intron_retain+=1

    #Count spliced isoforms based on strand
        if int(row[21])==0 and row[8] is '+': splice_pos+=1
        elif int(row[21])==0 and row[8] is '-': splice_neg+=1
        if int(row[21])==1 and row[8] is '+': retain_pos+=1
        elif int(row[21])==1 and row[8] is '-': retain_neg+=1    

    print directory
    print 'Spliced isoforms: '+str(spliced)
    print 'Intron-retaining isoforms: '+str(intron_retain)
    print '+ strand, spliced: '+str(splice_pos)
    print '- strand, spliced: '+str(splice_neg)
    print '+ strand, retaining: '+str(retain_pos)
    print '+ strand, retaining: '+str(retain_neg)

In [72]:
IsoformProductivity('2019-8-6_cdna/a431.b1/trcl_VA/nc.flair',
                       reads='/flair.collapse.isoforms.psl',
                       annot='software/flair/new.gencode.v29.annotation.gtf',
                       genome='2019-6-13_A431/GRCh38_trimmed.fa')

Unproductive proportion estimate 0.154203851574
2019-8-6_cdna/a431.b1/trcl_VA/nc.flair
Productive isoforms: 14724
Unproductive isoforms: 3283
Proportion of productive: 33%
lncRNA/other: 5316
Single exon transcripts: 20093


In [253]:
IntronRetention('2019-7-22_dirrna/trcl_VA/flair',
                   isoforms='isosproductive.psl',
                   outfile='isosproductive.ir.psl')

2019-7-22_dirrna/trcl_VA/flair
Spliced isoforms: 3060
Intron-retaining isoforms: 0
+ strand, spliced: 1603
- strand, spliced: 1457
+ strand, retaining: 0
+ strand, retaining: 0


In [268]:
#debug code to determine longest single exon transcript

f=open('2019-7-26_cdna/a431utc/trcl_VA/flair/isoforms.productivity.psl')
reader=csv.reader(f,delimiter='\t')
count=0
longest=0
printrow=''
for row in reader:
    if int(row[-1])==2 and int(row[0])>=300: 
        count+=1
        if int(row[0])>longest: longest=int(row[0])

print count
print 'longest single exon: '+str(longest)
f.close()
        
outfile='2019-7-26_cdna/a431.a1/trcl_VA/flair/isoforms.productivity.psl'
f=open(outfile)
reader=csv.reader(f,delimiter='\t')
count=0
for row in reader:
    if int(row[-1])==2 and int(row[0])>=300: 
        count+=1
        if int(row[0])>longest: longest=int(row[0])
print count
print 'longest single exon: '+str(longest)
f.close()

8588
longest single exon: 6696
8801
longest single exon: 8062


**Bedtools isoform-gene annotation using bedtools intersect function**

In [121]:
def Bedtools(directory,
             isoforms,     #can be a gtf or psl file
            annot,
            outfile):      #GTF file
    outfile=directory+'/'+outfile
    infile=directory+'/'+isoforms
    
    if 'psl' in isoforms:
        name=isoforms.split('.')[0]+'.gtf'
        !python software/flair/bin/psl_to_gtf.py {infile} > {directory}/{name}
        infile=directory+"/"+name
    
    !bedtools intersect -a {infile} -b {annot} -wb -s -split  > {outfile}
    f=open(outfile)
    reader=csv.reader(f,delimiter='\t')
    read_genes=[]
    transcript_dict={}
    overlap_transcripts=[]
    
    
    for row in reader:
        name=row[17].split('gene_name "')[1].split('";')[0]
        if name not in read_genes: read_genes.append(name)
            
        t=row[8].split('transcript_id "')[1].split('"')[0]
        if t not in transcript_dict: transcript_dict[t]=name
        if name not in transcript_dict[t] and name not in overlap_transcripts:
            overlap_transcripts.append(t)   
    
    f.seek(0)
    
    overlap_hits={}
    for row in reader:
        t=row[8].split('transcript_id "')[1].split('"')[0]
        name=row[17].split('gene_name "')[1].split('";')[0]
        if t in overlap_transcripts:
            if t in overlap_hits:
                overlap_hits[t].append(name)
            else:
                overlap_hits[t]=[]
                overlap_hits[t].append(name)
    
    for key in overlap_hits.keys():
        del transcript_dict[key]
        counts={}
        for gene in overlap_hits[key]:
            if gene in counts: counts[gene]+=1
            else: counts[gene]=1
        bestscore=0
        bestgene=''
        for key in counts:
            if counts[key]>bestscore:
                bestgene=key
                bestscore=counts[key]
        transcript_dict[t]=bestgene
    
    print 'Total genes expressed in '+directory+': '+str(len(read_genes))
    print 'Total transcripts: ' +str(len(transcript_dict))
    print str(len(overlap_transcripts))+' transcripts overlap multiple genes.'
    
    #Create isoform counts for every gene
    gene_counts={}
    for t in transcript_dict.keys():
        if transcript_dict[t] not in gene_counts: gene_counts[transcript_dict[t]]=1
        else: gene_counts[transcript_dict[t]]+=1
    
    #Create gene list
    import pandas as pd
    data=pd.DataFrame(list(gene_counts.items()))
    export_path="/scratch/tshishido/"+directory+"/isosproductive.gene_list.csv"
    export_csv=data.to_csv(export_path,index=None,header=True)

    print directory+' CHECK'
    print 'After correcting overlaps:'
    print 'Total transcripts: ' +str(len(transcript_dict))
    print 'Number of genes from transcript count: '+ str(len(gene_counts))
    
    counts=[]
    for gene in gene_counts.keys():
        counts.append(gene_counts[gene])
    largest=0
    for num in counts:
        if num>largest: largest=num
    plt.hist(counts,edgecolor='black',bins=20,range=[1,largest])
    plt.title('Isoform Distribution, '+directory)
    plt.xlabel('Number of Isoforms per Gene')
    plt.show()
    count_counts={}
    for num in counts:
        if num in count_counts: count_counts[num]+=1
        else: count_counts[num]=1
    print count_counts

In [122]:
Bedtools('2019-7-26_cdna/a431.b2/trcl_VA/nc.flair',
             'all_productive_isos.psl',
            annot='software/flair/exonsgenes.v29.annotation.gtf',
            outfile='productive.annotated')

***** WARNING: File software/flair/exonsgenes.v29.annotation.gtf has inconsistent naming convention for record:
chrM	ENSEMBL	exon	577	647	.	+	.	gene_id "ENSG00000210049.1"; transcript_id "ENST00000387314.1"; gene_type "Mt_tRNA"; gene_name "MT-TF"; transcript_type "Mt_tRNA"; transcript_name "MT-TF-201"; exon_number 1; exon_id "ENSE00001544501.1"; level 3; transcript_support_level "NA"; tag "basic";

***** WARNING: File software/flair/exonsgenes.v29.annotation.gtf has inconsistent naming convention for record:
chrM	ENSEMBL	exon	577	647	.	+	.	gene_id "ENSG00000210049.1"; transcript_id "ENST00000387314.1"; gene_type "Mt_tRNA"; gene_name "MT-TF"; transcript_type "Mt_tRNA"; transcript_name "MT-TF-201"; exon_number 1; exon_id "ENSE00001544501.1"; level 3; transcript_support_level "NA"; tag "basic";

Total genes expressed in 2019-7-26_cdna/a431.b2/trcl_VA/nc.flair: 7477
Total transcripts: 10182
22239 transcripts overlap multiple genes.
2019-7-26_cdna/a431.b2/trcl_VA/nc.flair CHECK
After correc

In [117]:
Bedtools('2019-7-26_cdna/a431.a1/trcl_VA/nc.flair',
             'all_productive_isos.psl',
            annot='software/flair/exonsgenes.v29.annotation.gtf',
            outfile='productive.annotated')

***** WARNING: File software/flair/exonsgenes.v29.annotation.gtf has inconsistent naming convention for record:
chrM	ENSEMBL	exon	577	647	.	+	.	gene_id "ENSG00000210049.1"; transcript_id "ENST00000387314.1"; gene_type "Mt_tRNA"; gene_name "MT-TF"; transcript_type "Mt_tRNA"; transcript_name "MT-TF-201"; exon_number 1; exon_id "ENSE00001544501.1"; level 3; transcript_support_level "NA"; tag "basic";

***** WARNING: File software/flair/exonsgenes.v29.annotation.gtf has inconsistent naming convention for record:
chrM	ENSEMBL	exon	577	647	.	+	.	gene_id "ENSG00000210049.1"; transcript_id "ENST00000387314.1"; gene_type "Mt_tRNA"; gene_name "MT-TF"; transcript_type "Mt_tRNA"; transcript_name "MT-TF-201"; exon_number 1; exon_id "ENSE00001544501.1"; level 3; transcript_support_level "NA"; tag "basic";

Total genes expressed in 2019-7-26_cdna/a431.a1/trcl_VA/nc.flair: 8520
Total transcripts: 13275
27375 transcripts overlap multiple genes.
2019-7-26_cdna/a431.a1/trcl_VA/nc.flair CHECK
After correc

In [29]:
Bedtools('2019-7-26_cdna/hpmouse1/nc.flair',
             'all_productive_isos.psl',
            annot='software/flair/new.gencode.v29.annotation.gtf',
            outfile='productive.annotated')

***** WARNING: File software/flair/new.gencode.v29.annotation.gtf has inconsistent naming convention for record:
chrM	ENSEMBL	gene	577	647	.	+	.	gene_id "ENSG00000210049.1"; gene_type "Mt_tRNA"; gene_name "MT-TF"; level 3;

***** WARNING: File software/flair/new.gencode.v29.annotation.gtf has inconsistent naming convention for record:
chrM	ENSEMBL	gene	577	647	.	+	.	gene_id "ENSG00000210049.1"; gene_type "Mt_tRNA"; gene_name "MT-TF"; level 3;

Total genes expressed in 2019-7-26_cdna/hpmouse1/nc.flair: 145
Total transcripts: 339
339 transcripts overlap multiple genes.
2019-7-26_cdna/hpmouse1/nc.flair CHECK
After correcting overlaps:
Total transcripts: 339
Number of genes from transcript count: 135
{1: 85, 2: 18, 3: 9, 4: 4, 5: 5, 6: 5, 7: 1, 9: 1, 10: 1, 11: 2, 14: 1, 17: 1, 20: 1, 21: 1}


**The number of transcripts accounted for in the output BAM generated by bedtools intersect is consistently smaller than the number of 

In [ ]:
#debug code for dealing with isoforms intersecting with multiple genes

outfile='2019-7-8_cdna/A431/trcl_VA/flair/isosproductive.annotated'
reader=csv.reader(open(outfile),delimiter='\t')
transcript_dict={}

for row in reader:
    t=row[8].split('transcript_id "')[1].split('"')[0]
    name=row[17].split('gene_name "')[1].split('";')[0]
    
    if t not in transcript_dict:
        transcript_dict[t]=[]
        transcript_dict[t].append(name)
    if t in transcript_dict:
        if name not in transcript_dict[t]: transcript_dict[t].append(name)

In [590]:
# attempt to debug transcripts intersecting multiple genes

file="2019-6-25_dirrna/trcl_VA/flair/isoforms.annotated"
f=open(file)
        
for x in overlap_transcripts:
    print x
    counts={}
    f.seek(0)     ##Resets reader to line 0
    reader=csv.reader(f,delimiter='\t')
    for row in reader:
        transcript=row[8].split('transcript_id "')[1].split('"')[0]
        gene=row[17].split('gene_name "')[1].split('"')[0]
        if transcript in x:
            if gene in counts:
                counts[gene]+=1
            else:
                counts[gene]=1
    print counts
        
    bestscore=0
    bestgene=''
    for gene in counts:
        if counts[gene]>bestscore:
            bestgene=gene
            bestscore=counts[gene]
    print x+', best gene: '+bestgene+' '+str(bestscore)

b0428187-4017-4336-bc0a-92587948592f_NC
{'CROCCP2': 6}
b0428187-4017-4336-bc0a-92587948592f_NC, best gene: CROCCP2 6
b243ea2c-f568-4cd4-820c-c88cc6a9e6e4
{'PPIE': 93}
b243ea2c-f568-4cd4-820c-c88cc6a9e6e4, best gene: PPIE 93
8a25c52d-a31f-4f3f-a898-afdd6bc9b903
{'MTF2': 100}
8a25c52d-a31f-4f3f-a898-afdd6bc9b903, best gene: MTF2 100
0c8ceeaf-b971-4aec-9cb1-c42553b77d75_NC
{'RBBP4': 2}
0c8ceeaf-b971-4aec-9cb1-c42553b77d75_NC, best gene: RBBP4 2
62dc2f3b-13e8-4c71-b26e-2c66480d2f39_NC
{'ATF6': 2}
62dc2f3b-13e8-4c71-b26e-2c66480d2f39_NC, best gene: ATF6 2
10f56ffc-ffc0-4d54-bf61-801a35569424_NC
{'PKN2': 2}
10f56ffc-ffc0-4d54-bf61-801a35569424_NC, best gene: PKN2 2
bd8f3e68-4560-42df-aac3-cbffde29181c
{'PTPRU': 133}
bd8f3e68-4560-42df-aac3-cbffde29181c, best gene: PTPRU 133
f99df470-176f-4317-938a-2602a619a46d
{'LINC00623': 5}
f99df470-176f-4317-938a-2602a619a46d, best gene: LINC00623 5
f77e34b7-c5fa-467f-8f21-011a248e8eda_NC
{'DCAF6': 6}
f77e34b7-c5fa-467f-8f21-011a248e8eda_NC, best gene: D

KeyError: 'TBCE'

**Other notes**

In [ ]:
#Other notes

#Created fasta reference genome for only human chr.21
!sed -n "/^>NC_000021.9$/,/^>/p" software/homo_GRCh38_trimmed_ref.fa >  software/homo_chr21.GRCh38_trimmed_ref.fa

#Created human GENCODE annotation file of only exon and gene entries, to use for bedtools intersect
!grep '	gene	' software/flair/new.gencode.v29.annotation.gtf > software/flair/genes.v29.annotation.gtf 
!grep '	exon	' software/flair/new.gencode.v29.annotation.gtf > software/flair/exons.v29.annotation.gtf
!cat software/flair/exons.v29.annotation.gtf software/flair/genes.v29.annotation.gtf > software/flair/exonsgenes.v29.annotation.gtf


#Did not get around to using these 2 flair scripts but could be of good use for 
    ##identifying differential isoform expression
    ##1)
count_file='2019-7-26_cdna/a431.a1/trcl_VA/flair/counts_matrix.tsv'
out_dir='2019-7-26_cdna/a431.a1/trcl_VA/flair/diffexp'
!python software/flair/flair.py diffExp -q {count_file} -o {out_dir}
    ##2)
#python diff_iso_usage.py isoforms.psl colnum1 colnum2 diff_isos.txt
isoforms='2019-7-8_cdna/SHSY5Y/trcl_VA/flair/3.flair.collapse.isoforms.psl' 
outfile='2019-7-8_cdna/SHSY5Y/trcl_VA/flair/diff_isos.txt'
!python software/flair/bin/diff_iso_usage.py {isoforms} {outfile}

#Bedtools annotate function, did not look into
annot='software/flair/exonsgenes.v29.annotation.gtf'
isoforms='2019-7-8_cdna/A431/trcl_VA/flair/3.flair.collapse.isoforms.gtf'
output="2019-7-8_cdna/A431/trcl_VA/flair/bedtools.annotate"
!bedtools annotate -i {annot} -files {isoforms} > {output}

#This flair script didn't do anything, just reappended on the chromosome position
isoforms='2019-7-8_cdna/SHSY5Y/trcl_VA/flair/exonsgenes.collapse.isoforms.psl' 
outfile='2019-7-8_cdna/SHSY5Y/trcl_VA/flair/exonsgenes.named'
annot='software/flair/new.gencode.v29.annotation.gtf'
!python software/flair/bin/identify_gene_isoform.py {isoforms} {annot} {outfile}

#Flair script identifies gene annotation for each entry of PSL. Only corrected a fraction of the isoforms
    ##missing genes annotations

psl='2019-7-8_cdna/SHSY5Y/trcl_VA/flair/exonsgenes.collapse.isoforms.psl'
ref='software/flair/new.gencode.v29.annotation.gtf'
outfile='2019-7-8_cdna/SHSY5Y/trcl_VA/flair/isos_matched.psl'
!python software/flair/bin/identify_annotated_gene.py {psl} {ref} {outfile}
